In [ ]:
!pip install python-dotenv
!pip install httpx==0.27.2
!pip install anthropic
!pip install gradio

In [ ]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

import anthropic
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import google.generativeai

import json
import gradio as gr

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving API Key.env to API Key (1).env


In [ ]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('CLAUDE_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')

check_api_open = f"OpenAI API Key exists and begins {openai_api_key[:7]}" if openai_api_key else "OpenAI API Key not set"
check_api_anthropic = f"Anthropic API Key exists and begins {anthropic_api_key[:7]}" if anthropic_api_key else "Anthropic API Key not set"
check_api_gemini = f"Google API Key exists and begins {google_api_key[:8]}" if google_api_key else "Google API Key not set"

display(check_api_open, check_api_anthropic, check_api_gemini)

'OpenAI API Key exists and begins sk-proj'

'Anthropic API Key exists and begins sk-ant-'

'Google API Key exists and begins AIzaSyCH'

In [ ]:
openai = OpenAI(api_key = openai_api_key)
claude = anthropic.Anthropic(api_key=anthropic_api_key)
google.generativeai.configure(api_key = google_api_key)

In [ ]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "Da nang": "$1000",
                 "beijing": "$500", "shanghai": "$800"}

def get_price(city):
  return ticket_prices.get(city.lower())

In [ ]:
def get_person(city):
    response = response = openai.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content": "You are an assistant to return the famous person of this city."},
        {"role": "user", "content": f"let me know the person got more fame of this {city}"}
    ]
    )
    return response.choices[0].message.content

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_price",
            "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city that the customer wants to travel to",
                    },
                },
                "required": ["city"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_person",
            "description": "get famous person of this city, Return the weather, such as information of this person.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "the most person of this city"
                    }
                },
                "required": ["city"],
                "additionalProperties": False
            }
        }
    }
  ]

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 3 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
import base64
from io import BytesIO
from PIL import Image

def artist(city):
  image_response = openai.images.generate(
      model = "dall-e-3",
      prompt = f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
      size = "1024x1024",
      n = 1,
      response_format = "b64_json"
  )

  image_data = image_response.data[0].b64_json
  image_bytes = base64.b64decode(image_data)
  image = Image.open(BytesIO(image_bytes))

  return image

In [ ]:
def translate_text(message, target_language="Vietnamese"):
    try:
        response = claude.messages.create(
            model="claude-3-haiku-20240307",
            max_tokens=1000,
            temperature=0.7,
            system="You are a translator. Translate it to Vietnamese",
            messages=[
                {"role": "user", "content": message},
            ],
        )

        if response.content and len(response.content) > 0:
            return response.content[0].text
        else:
            return "Translation error: Empty response from Claude"
    except Exception as e:
        return f"Translation error: {str(e)}"

In [ ]:
def chat_bot(message, history, translation_history):
    formatted_history = [{"role": msg.get("role"), "content": msg.get("content")} for msg in history]

    messages = [
        {"role": "system", "content": system_message}
    ]
    messages.extend(formatted_history)
    messages.append({"role": "user", "content": message})

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools
    )
    image = None
    if response.choices[0].finish_reason == "tool_calls":
        available_functions = {
            "get_price": get_price,
            "get_person": get_person
        }
        messages.append(response.choices[0].message)
        tool_calls = response.choices[0].message.tool_calls

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            if function_name in available_functions:
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)

                city = function_args.get("city")

                if not city:
                    raise ValueError("City not provided in function arguments")

                try:
                    result = function_to_call(city)
                except Exception as e:
                    result = f"Error calling function {function_name}: {str(e)}"

                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": str(result)
                })

            if function_name == "get_price":
                image = artist(city)

        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )


    assistant_response = response.choices[0].message.content
    history.append({"role": "assistant", "content": assistant_response})

    translation_history.append({"role": "assistant", "content": translate_text(assistant_response)})

    return history, translation_history, image

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, label="Original Chat", type="messages")
        translation_chatbot = gr.Chatbot(height=500, label="Translated Chat", type="messages")
    with gr.Row():
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history, translation_history):
        history += [{"role": "user", "content": message}]
        translation_history += [{"role": "user", "content": translate_text(message)}]
        return "", history, translation_history

    entry.submit(
        do_entry,
        inputs=[entry, chatbot, translation_chatbot],
        outputs=[entry, chatbot, translation_chatbot]
    ).then(
        chat_bot,
        inputs=[entry, chatbot, translation_chatbot],
        outputs=[chatbot, translation_chatbot, image_output]
    )

    def clear_all():
        return None, None

    clear.click(clear_all, inputs=None, outputs=[chatbot, translation_chatbot], queue=False)

ui.launch(inbrowser=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e6f77204f8ed8e3f01.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
